In [1]:
from sklearn.neighbors import KNeighborsClassifier
import csv
import pickle
import re
from datetime import datetime
import numpy as np
# randn = np.random.randn
from pandas import *
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.stats import linregress
from pylab import *
from urllib import urlopen
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

# Poisson GLM Challenges

## Data for the Challenges

We are going to fit a model to count data. The first one we will use is about the total count of damage incidents to ships. You can get it here: http://data.princeton.edu/wws509/datasets/ships.dta

We are stealing this dataset from STATA, so it's in STATA format. Pandas has a reader for that format to make your life easy:

from pandas.io.stata import StataReader
reader = StataReader('ships.dta')
dataframe = reader.data()
Here are some details on this dataset:

The file has 34 rows corresponding to the observed combinations of type of ship, year of construction and period of operation. Each row has information on five variables as follows:

ship type, coded 1-5 for A, B, C, D and E,
year of construction (1=1960-64, 2=1965-70, 3=1970-74, 4=1975-79),
period of operation (1=1960-74, 2=1975-79)
months of service, ranging from 63 to 20,370, and
damage incidents, ranging from 0 to 53.
Note that there no ships of type E built in 1960-64, and that ships built in 1970-74 could not have operated in 1960-74. These combinations are omitted from the data file.



## Challenge 1

Model the damage incident counts with a Poisson Regression.

Hint: You can look at the previous ipython notebook with the logistic GLM example to see how you can do GLM with statsmodels
Remember that you will have to create dummy variables for categorical variables, and if you have time bins (like "1960-1964"), you have the option of either a) treating each bin as a category (and create dummies for each bin), or b) treat it as a continuous variable and take the mid-value (1962). Also remember to add a constant (to model the intercept).

Take a look at the statsmodels summary table, the goodness of fit indicators (Deviance, Pearson's chi square statistic) and the coefficients. Is this a good model?

## Challenge 2

The months of service provides the time interval in which a ship has chances to acquire damages. It can be thought of "exposure", and this column can be used as an offset. You can check these two resources for a quick idea on exposure and using an offset: Offset in Wikipedia When to use an offset in CrossValidated (StackOverflow for statistics)

Try your model with months of service as the offset. Does it perform better?

## Challenge 3

Now separate your data (even though it's only 14 rows) into a training and test set (your test will only be 4 or 5 rows), and check if you predict well (you can look at mean absolute error or mean squared error using sklearn.metrics).

## Challenge 4

Deviance. Compute the difference in Deviance statistics for your model and the null model. This is called the null deviance. You can do this in one of 2 ways:

We need the deviance for the null model (a model where none of the explanatory variables are used; it's just a model with a mean guess). To do that, fit a poisson regression with only a constant. Get the deviance for this null model. Take the difference of deviances between your model and this null model.
Use statsmodels.genmod.generalized_linear_model.GLMResults
Check if this difference is extreme enough that we can reject the null hypothesis. If we can't reject the null hypothesis, we cannot say that this model tells us more than that trivial, null model. To calculate the p-value (prob. of getting a deviance difference at least as extreme as this under the null hypothesis), we need to do a hypothesis test.

You can import the chi square test from scipy for this:

from scipy.stats import chisqprob
Is your model better than the null model?

## Challenge 5

Now, instead of a poisson regression, do an ordinary least squares regression with log Y. Compare the models. Are the coefficients close? Do they perform similarly?

## Challenge 6

Now, let's do this on another dataset: Smoking and Cancer.

You can get it here: http://data.princeton.edu/wws509/datasets/smoking.dta This dataset has information on lung cancer deaths by age and smoking status. It has these columns:

age: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
smoking status: coded 1 = doesn't smoke, 2 = smokes cigars or pipe only, 3 = smokes cigarrettes and cigar or pipe, and 4 = smokes cigarrettes only,
population: in hundreds of thousands, and
deaths: number of lung cancer deaths in a year.
That population looks a lot like an offset!

Fit poisson and OLS models and compare them.

Bonus: try all these things in R! Your numerical answers should all be the same.

In [2]:
# Load modules and data
import statsmodels.api as sm
data = sm.datasets.scotland.load()
data.exog = sm.add_constant(data.exog)
type(data.endog)

numpy.ndarray

In [3]:
# Instantiate a gamma family model with the default link function.
gamma_model = sm.GLM(data.endog, data.exog, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
gamma_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   32
Model:                            GLM   Df Residuals:                       24
Model Family:                   Gamma   Df Model:                            7
Link Function:          inverse_power   Scale:                0.00358428317349
Method:                          IRLS   Log-Likelihood:                -83.017
Date:                Sat, 09 May 2015   Deviance:                     0.087389
Time:                        00:08:13   Pearson chi2:                   0.0860
No. Iterations:                     6                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.0178      0.011     -1.548      0.122        -0.040     0.005
x1          4.962e-05   1.62e-05      3.060      0.002      1.78e-05  8.14e-05
x2             0.0020      0.001      3.824      0.000         0.001     0.003
x3         -7.181e-05   2.71e-05     -2.648      0.008        -0.000 -1.87e-05
x4             0.0001   4.06e-05      2.757      0.006      3.23e-05     0.000
x5         -1.468e-07   1.24e-07     -1.187      0.235     -3.89e-07  9.56e-08
x6            -0.0005      0.000     -2.159      0.031        -0.001 -4.78e-05
x7         -2.427e-06   7.46e-07     -3.253      0.001     -3.89e-06 -9.65e-07
==============================================================================
"""

In [4]:
from pandas.io.stata import StataReader
reader = StataReader('ships.dta')
ships = reader.data()

In [5]:
ships.head(5)

,type,construction,operation,months,damage
0,A,1960-64,1960-74,127,0
1,A,1960-64,1975-79,63,0
2,A,1965-70,1960-74,1095,3
3,A,1965-70,1975-79,1095,4
4,A,1970-74,1960-74,1512,6


In [6]:
constdummy = []
for n in ships['construction']:
    constdummy.append((int(mean([int(n[2:4]), int(n[5:7])])))+1900)
ships['constdummy'] = constdummy
ships.head(5)

,type,construction,operation,months,damage,constdummy
0,A,1960-64,1960-74,127,0,1962
1,A,1960-64,1975-79,63,0,1962
2,A,1965-70,1960-74,1095,3,1967
3,A,1965-70,1975-79,1095,4,1967
4,A,1970-74,1960-74,1512,6,1972


In [7]:
opdummy = []
for n in ships['operation']:
    opdummy.append((int(mean([int(n[2:4]), int(n[5:7])])))+1900)
ships['opdummy'] = opdummy
ships.head(5)

,type,construction,operation,months,damage,constdummy,opdummy
0,A,1960-64,1960-74,127,0,1962,1967
1,A,1960-64,1975-79,63,0,1962,1977
2,A,1965-70,1960-74,1095,3,1967,1967
3,A,1965-70,1975-79,1095,4,1967,1977
4,A,1970-74,1960-74,1512,6,1972,1967


In [8]:
X = ships.drop(['construction', 'operation'], 1)
X = np.asarray(X)
y = ships['damage']
y = np.asarray(y)
y = sm.add_constant(y)

In [9]:
ships_model = sm.GLM(y, X, family=sm.families.Gamma())
ships_results = ships_model.fit()
ships_results.summary()

TypeError: unsupported operand type(s) for -: 'unicode' and 'unicode'